In [2]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt

In [4]:
import seaborn as sns

In [5]:
df=pd.read_csv("C:/Users/HP/Documents/BDMDM/Apparel_Shopping_Dataset.csv")

In [6]:
df.head()

,C_NO,V1,V2,V3,V4,V5,V6,Income,Mall Visits
0,1,6,4,7,3,2,3,65000,4
1,2,2,3,1,4,5,4,43000,1
2,3,7,2,6,4,1,3,58000,3
3,4,4,6,4,5,3,6,26000,6
4,5,1,3,2,2,6,4,45000,1


In [7]:
df.describe()

,C_NO,V1,V2,V3,V4,V5,V6,Income,Mall Visits
count,20.00000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000
mean,10.50000,3.850000,4.100000,3.950000,4.100000,3.450000,4.350000,45825.00000,3.400000
std,5.91608,1.899446,1.410487,2.012461,1.518309,1.761429,1.496487,14615.48836,1.902906
min,1.00000,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000,26000.00000,1.000000
25%,5.75000,2.000000,3.000000,2.000000,3.000000,2.000000,3.000000,34125.00000,1.750000
50%,10.50000,4.000000,4.000000,4.000000,4.000000,3.500000,4.000000,42000.00000,3.500000
75%,15.25000,5.250000,5.000000,6.000000,5.250000,4.250000,5.250000,55750.00000,5.000000
max,20.00000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,71000.00000,7.000000


[ "###### Loading required packages and normalizing the clustering variables.\n", "\n", "###### As our objective is to generate the customer groups based on the psychographic attributes. We will use variables V1, V2, V3, V4, V5, V6 for clustering algorithm.\n", "\n", "###### K-means algortithm works better if normalize the variables before applying clustering. So, we will create a copy of dataframe and store the normalized variable scores in it. We will transform these variables such that each variable will have mean value of 0 and standard deviation on 1. This is necessary step to bring all the attributes on the similar scale. If clustering attributes have different range and scale, some variable will get more importance in distance computation, which will have adverse effect on the clustering." ] 

In [8]:
# Scaling data 
from sklearn.preprocessing import StandardScaler 
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_score
df_norm=df[:]
scaler = StandardScaler()
scaler.fit(df_norm[['V1','V2','V3','V4','V5','V6']])
df_norm[['V1','V2','V3','V4','V5','V6']] = scaler.transform(df_norm[['V1','V2','V3','V4','V5','V6']])
# df_rfm_normal = pd.DataFrame(df_rfm_normal, index=df_rfm_log.index, columns=df_rfm_log.columns)
# Check result after standardising
df_norm.describe().round(3)

,C_NO,V1,V2,V3,V4,V5,V6,Income,Mall Visits
count,20.000,20.000,20.000,20.000,20.000,20.000,20.000,20.000,20.000
mean,10.500,-0.000,0.000,-0.000,0.000,-0.000,0.000,45825.000,3.400
std,5.916,1.026,1.026,1.026,1.026,1.026,1.026,14615.488,1.903
min,1.000,-1.539,-1.528,-1.504,-1.419,-1.427,-1.611,26000.000,1.000
25%,5.750,-0.999,-0.800,-0.994,-0.743,-0.845,-0.926,34125.000,1.750
50%,10.500,0.081,-0.073,0.025,-0.068,0.029,-0.240,42000.000,3.500
75%,15.250,0.756,0.655,1.045,0.777,0.466,0.617,55750.000,5.000
max,20.000,1.701,2.109,1.555,1.960,2.068,1.817,71000.000,7.000


###### Finding optimal number of k, i.e. number of clusters to be created from dataset. Value of k with the highest silhouette score can be considered as optimal number of clusters to be generated. We can also look at with in cluster variance grpah to identify the elbow point.
##### Here, SS indicates Silhouette score and intertia indicates wihtin cluster variance.

In [18]:
def optimal_kmeans(dataset, start=2, end=11):
    '''
    Calculate the optimal number of kmeans
    
    INPUT:
        dataset : dataframe. Dataset for k-means to fit
        start : int. Starting range of kmeans to test
        end : int. Ending range of kmeans to test
    OUTPUT:
        Values and line plot of Silhouette Score.
    '''
    
    # Create empty lists to store values for plotting graphs
    n_clu = []
    km_ss = []
    inertia = []

    # Create a for loop to find optimal n_clusters
    for n_clusters in range(start, end):

        # Create cluster labels
        kmeans = KMeans(n_clusters=n_clusters)
        labels = kmeans.fit_predict(dataset)

        # Calcualte model performance
        silhouette_avg = round(silhouette_score(dataset, labels, random_state=1), 3)
        inertia_score = round(kmeans.inertia_, 2) 

        # Append score to lists
        km_ss.append(silhouette_avg)
        n_clu.append(n_clusters)
        #inertia.append(inertia_score)

        print("No. Clusters: {}, Silhouette Score: {},  SS Delta: {}, Inertia: {}, , Inertia Delta: {}".format(
            n_clusters, 
            silhouette_avg, 
            (km_ss[n_clusters - start] - km_ss[n_clusters - start - 1]).round(3))) 
            #inertia_score,
            #(inertia[n_clusters - start] - inertia[n_clusters - start - 1]).round(3)))
        

        # Plot graph at the end of loop
        if n_clusters == end - 1:
            plt.figure(figsize=(9, 6))
            #plt.subplot(2, 1, 2)
            #plt.title('Within-Cluster Sum-of-Squares / Inertia')
            sns.pointplot(x=n_clu, y=inertia)
            #plt.subplot(2, 1, 2)
            plt.title('Silhouette Score')
            sns.pointplot(x=n_clu, y=km_ss)
            plt.tight_layout()
        plt.show()

In [19]:
optimal_kmeans(df[['V1','V2','V3','V4','V5','V6']])

IndexError: Replacement index 3 out of range for positional args tuple

In [21]:
kmeans = KMeans(n_clusters = 3,random_state=3,init='k-means++',n_init=10)
kmeans.fit(df_norm[['V1','V2','V3','V4','V5','V6']])

KMeans(n_clusters=3, random_state=3)

In [22]:
kmeans.labels_



array([1, 0, 1, 2, 0, 1, 1, 1, 0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 2, 0])

In [23]:
df['Cluster_Label']=kmeans.labels_+1

In [24]:
df

,C_NO,V1,V2,V3,V4,V5,V6,Income,Mall Visits,Cluster_Label
0,1,6,4,7,3,2,3,65000,4,2
1,2,2,3,1,4,5,4,43000,1,1
2,3,7,2,6,4,1,3,58000,3,2
3,4,4,6,4,5,3,6,26000,6,3
4,5,1,3,2,2,6,4,45000,1,1
5,6,6,4,6,3,3,4,55000,4,2
6,7,5,3,6,3,3,4,67000,3,2
7,8,7,3,7,4,1,4,70000,5,2
8,9,2,4,3,3,6,3,39000,1,1
9,10,3,5,3,6,4,6,35000,5,3


In [25]:
#df.to_excel(\"D:/courses/BDMDM 2020/Session 5/Apparel_Shopping_Dataset.xlsx\")

In [26]:
df.groupby(['Cluster_Label']).agg(
    { 'V1': 'mean',
     'V2': 'mean',
     'V3': 'mean',
     'V4': 'mean',
     'V5': 'mean',
     'V6': 'mean',
     'Income':'mean',
     'Mall Visits': 'mean'})

,V1,V2,V3,V4,V5,V6,Income,Mall Visits
Cluster_Label,,,,,,,,
1,1.666667,3.000000,1.833333,3.500,5.500,3.333333,41000.000000,1.166667
2,5.750000,3.625000,6.000000,3.125,1.875,3.875000,61250.000000,3.500000
3,3.500000,5.833333,3.333333,6.000,3.500,6.000000,30083.333333,5.500000


In [27]:
#[ "def snake_plot(df):\n", " '''\n", " Transform dataframe and plot snakeplot\n", " '''\n", " \n", " # Melt data into long format\n", " df_melt=pd.melt(df.reset_index(), \n", " id_vars=['Cluster_Label'],\n", " value_vars=['V1', 'V2', 'V3','V4','V5','V6'], \n", " var_name='Metric', \n", " value_name='Value')\n", "\n", " plt.xlabel('Metric')\n", " plt.ylabel('Value')\n", " sns.pointplot(data=df_melt,x='Metric', y='Value', hue='Cluster_Label')\n", " print(df_melt)\n", " return" ]

In [28]:
#plt.subplot(1, 1, 1)\n", "plt.title('Snake Plot of K-Means = 3')\n", "snake_plot(df)" ]

In [29]:
#[ "df.groupby('Cluster_Label').agg( \n", " Mean_V1=('V1','mean'), \n", " Min=('V1',min),\n", " Max=('V1',max))" ] 

In [30]:
#[ "# Hierarchical clustering\n", "import scipy.cluster.hierarchy as shc\n", "from sklearn.cluster import AgglomerativeClustering\n", "from kmodes.kprototypes import KPrototypes\n", "from kmodes.kmodes import KModes" ]

In [31]:
#[ "plt.figure(figsize=(10, 7)) \n", "plt.title(\"Dendrograms\") \n", "dend = shc.dendrogram(shc.linkage(df[['V1','V2','V3','V4','V5','V6']], method='ward'))\n", "plt.axhline(y=7.5, color='r', linestyle='--')" ]

In [32]:
#[ "hier = AgglomerativeClustering(n_clusters = 4, affinity='euclidean', linkage='ward')\n", "hier.fit(df_norm)\n", "# Linkage methods: single, average, ward, complete" ]

In [33]:
#[ "# Extract cluster labels\n", "df['Hier_Clust_Labels']=hier.labels_+1" ]

In [34]:
#[ "df.groupby(\n", " ['Hier_Clust_Labels']\n", ").agg(\n", " {\n", " 'V1': 'mean', # Average height for each sport\n", " 'V2': 'mean', # get the number of sports country is participating in\n", " 'V3': 'mean', # get the first year of country participation\n", " 'V4': 'mean',\n", " 'V5': 'mean', # numbe of unique players participated in olympic\n", " 'V6': 'mean',\n", " 'Income': 'mean',\n", " 'Mall Visits': 'mean'\n", " }\n", ")" ]

In [35]:
#[ "df" ] 

In [36]:
# [ "from sklearn.cluster import DBSCAN\n", "clustering = DBSCAN(eps=3, min_samples=4)\n", "clustering.fit(df[['V1','V2','V3','V4','V5','V6']])" ] 

In [37]:
#[ "clustering.labels_" ]

In [38]:
#df['DBSCAN_Clust_Labels']=clustering.labels_+1

In [39]:
#df.groupby(\n", " ['DBSCAN_Clust_Labels']\n", ").agg(\n", " {\n", " 'V1': ['mean','count'], # Average height for each sport\n", " 'V2': 'mean', # get the number of sports country is participating in\n", " 'V3': 'mean', # get the first year of country participation\n", " 'V4': 'mean',\n", " 'V5': 'mean', # numbe of unique players participated in olympic\n", " 'V6': 'mean',\n", " 'Income': 'mean',\n", " 'Mall Visits': 'mean'\n", " }\n", ")" ]

In [ ]:
#df